In [2]:
# Step 1: Mount Google Drive (optional if you want to save the model there)
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import shutil

gdrive_path = "/content/drive/My Drive/grabcut_segmented_dataset.zip"
local_path = "grabcut_segmented_dataset.zip"

# Copy zip to local environment for faster access
shutil.copy(gdrive_path, local_path)



'grabcut_segmented_dataset.zip'

In [13]:
# Step 3: Unzip dataset
import zipfile

with zipfile.ZipFile(local_path, 'r') as zip_ref:
    zip_ref.extractall("data")

dataset_dir = "data/grabcut_segmented_dataset"


In [ ]:
import os
import shutil

def reorganize_by_front_back(base_dir):
    for coin_class in os.listdir(base_dir):
        class_path = os.path.join(base_dir, coin_class)
        if not os.path.isdir(class_path):
            continue

        # Create front and back subfolders
        front_dir = os.path.join(class_path, "front")
        back_dir = os.path.join(class_path, "back")
        os.makedirs(front_dir, exist_ok=True)
        os.makedirs(back_dir, exist_ok=True)

        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)
            if os.path.isfile(file_path):
                lowercase_name = filename.lower()
                if "front" in lowercase_name:
                    shutil.move(file_path, os.path.join(front_dir, filename))
                elif "back" in lowercase_name:
                    shutil.move(file_path, os.path.join(back_dir, filename))

# Apply to dataset directory
reorganize_by_front_back(dataset_dir)


In [14]:
# Step 4: Create image dataset from folder
import tensorflow as tf

# Flatten subfolders (front/back) under each class
def flatten_and_reorganize(dataset_dir):
    import shutil
    import os
    import glob

    flat_dir = "flat_dataset"
    os.makedirs(flat_dir, exist_ok=True)

    for coin_class in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, coin_class)
        if os.path.isdir(class_path):
            for view in ["front", "back"]:
                view_path = os.path.join(class_path, view)
                if os.path.exists(view_path):
                    target_class_dir = os.path.join(flat_dir, coin_class)
                    os.makedirs(target_class_dir, exist_ok=True)
                    for img_file in glob.glob(os.path.join(view_path, "*")):
                        shutil.copy(img_file, target_class_dir)
    return flat_dir

flat_dataset_path = flatten_and_reorganize(dataset_dir)


In [15]:
# Step 5: Load the images as tf.data.Dataset
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    flat_dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    flat_dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


Found 4721 files belonging to 7 classes.
Using 3777 files for training.
Found 4721 files belonging to 7 classes.
Using 944 files for validation.
Classes: ['10c', '20c', '50c', '5c', 'R1', 'R2', 'R5']


In [16]:
# Step 6: Optimize dataset performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [17]:
# Step 7: Build a simple CNN
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,630,247 (6.22 MB)

 Trainable params: 1,630,247 (6.22 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Step 8: Train the model
EPOCHS = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)


Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.3870 - loss: 1.6187 - val_accuracy: 0.5699 - val_loss: 1.1695
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6027 - loss: 1.0353 - val_accuracy: 0.6441 - val_loss: 0.9533
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7135 - loss: 0.7800 - val_accuracy: 0.6780 - val_loss: 0.9168
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7893 - loss: 0.5909 - val_accuracy: 0.7055 - val_loss: 0.8311
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8629 - loss: 0.4150 - val_accuracy: 0.7553 - val_loss: 0.7918
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9159 - loss: 0.2701 - val_accuracy: 0.7627 - val_loss: 0.8021
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9332 - loss: 0.2228 - val_accuracy: 0.7850 - val_loss: 0.8055
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9480 - loss: 0.1596 - val_accuracy: 0

In [19]:
# Step 9: Convert to TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("real_time_coin_detector(grabcut).tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpx42v8f39'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  138921603092112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921603091344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921603088464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921603089040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921603091920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921603091152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921593169552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921593168592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921593168976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138921593173008: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [20]:
# Step 10: Download the TFLite model
from google.colab import files
files.download("real_time_coin_detector(grabcut).tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>